In [11]:
from tkinter import *
from PIL import Image, ImageDraw
import math
import itertools

# PyTorch imports
import torch as tc

# CUDA
if tc.cuda.is_available():
    device = tc.device("cuda")
else:
    device = tc.device("cpu")

device = tc.device("cpu")

print(f"CUDA is available: {tc.cuda.is_available()}")

stroke = []
stroke_list = []


def perpendicularDistance(point, line):
    x0, y0 = point
    x1, y1 = line[0]
    x2, y2 = line[1]

    # Calculate the perpendicular distance
    return abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1) / math.sqrt((y2 - y1)**2 + (x2 - x1)**2)


def Line(point1, point2):
    return point1, point2


# source: https://karthaus.nl/rdp/
def DouglasPeucker(PointList, epsilon):
    # Find the point with the maximum distance
    dmax = 0
    index = 0
    end = len(PointList)
    for i in range(1, end - 1):
        d = perpendicularDistance(PointList[i], Line(PointList[0], PointList[end - 1]))
        if d > dmax:
            index = i
            dmax = d

    ResultList = []

    # If max distance is greater than epsilon, recursively simplify
    if dmax > epsilon:
        # Recursive call
        recResults1 = DouglasPeucker(PointList[:index + 1], epsilon)
        recResults2 = DouglasPeucker(PointList[index:], epsilon)

        # Build the result list
        ResultList = recResults1[:-1] + recResults2
    else:
        ResultList = [PointList[0], PointList[end - 1]]

    # Return the result
    return ResultList


def get_xy(event):
    global x, y
    (x, y) = (event.x, event.y)


def draw(event):
    global x, y
    canvas.create_line((x, y, event.x, event.y), fill='white')
    (x, y) = (event.x, event.y)
    stroke.append((x, y))


def save_stroke(event, epsilon=2.0):
    global stroke
    global canvas
    global img
    
    stroke = DouglasPeucker(stroke, epsilon)
    stroke_list.append(stroke)
    ImageDraw.Draw(img).line(stroke, fill=1, width=1)
    img.save("canvas.png")
    stroke = []


CUDA is available: True


In [2]:
if __name__ == "__main__":

    app = Tk()
    app.geometry("256x256")

    canvas = Canvas(app, bg='black')
    canvas.pack(anchor='nw', fill = 'both', expand=1)

    img = Image.new('1', (256, 256), color=0)

    canvas.bind("<Button-1>", get_xy)
    canvas.bind("<B1-Motion>", draw)
    canvas.bind("<ButtonRelease-1>", save_stroke)

    app.mainloop()

In [66]:
def get_category(line_idx, txt_path):
    with open(txt_path) as f:
                line = itertools.islice(f, line_idx, line_idx+1)
                line = map(lambda s: s.strip(), line)
                return list(line)[0]

if __name__ == "__main__":

    with tc.no_grad():

        filepath = './'
        filename = "Wonky_Doodles_CNN.pth"
        model = tc.load(f"{filepath}{filename}").to(device)

        img = Image.open('canvas.png')
        tns = transforms.ToTensor()(img).to(device)
        otpt = tc.softmax(model(tns)[0], dim = 0)

        prob_list = []
        for idx, category in enumerate(otpt):
            # print(f"{get_category(idx, 'label_list.txt')} {100 * category:.2f} %")
            prob_list.append((get_category(idx, 'label_list.txt'), round(100 * category.item(), 2)))
            if idx > 9: break
        prob_list.sort(key=lambda x: x[1], reverse=True)
        for category in prob_list:
            print(f"{category[0]}: {category[1]} %")




banana: 98.1 %
aircraft carrier: 1.15 %
bandage: 0.75 %
